In [ ]:
#1. 라이브러리 호출

from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

# 2. 크롤링 시 필요한 함수 정의

In [ ]:

# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  # 입력된 수를 1, 11, 21, 31 ...로 변환
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

In [ ]:

# url 생성 함수 (검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls



In [ ]:
# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

# 3. 크롤링할 네이버 뉴스 URL 추출

In [ ]:
#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")


In [ ]:
# naver url 생성
url = makeUrl(search,page,page2)

In [ ]:
#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)

In [ ]:
#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

In [ ]:
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

In [ ]:


#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

검색할 키워드를 입력해주세요:AI

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):3

크롤링할 종료 페이지:  3 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=AI&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=AI&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=AI&start=21']


100%|██████████| 59/59 [00:00<00:00, 364991.06it/s]


# 4.뉴스 본문 및 날짜 크롤링

In [ ]:
for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")

    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date)) # Indentation fixed here
    # 날짜 가져오기
    news_dates.append(news_date)

In [ ]:
print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

100%|██████████| 29/29 [00:30<00:00,  1.04s/it]

검색된 기사 갯수: 총  30 개

[뉴스 제목]
['SKT, 美 AI 데이터센터 솔루션 기업에 2800억 투자…"역대 AI 투자 중 최대"(종합)', '정부, AI 근로감독관 등 초거대 AI 공공 서비스 개발 착수', "곽노정 SK하이닉스 사장, 美 현지서 'AI 반도체 협력' 논의", '배민 ‘AI 추천배차’, 라이더 사고 확률 27.8% 감소시켰다', '이글루, 리벨리온과 AI 보안 어시스턴트 개발한다', "신한운용, 'SOL 미국AI 전력인프라 ETF' 상장", '한컴, AI 명령어 템플릿으로 서울시 행정업무 효율화 제안', '강민수 국세청장 후보 "악의적 탈세 단호히 대처…AI기술 접목"', '“일자리 90%, 6년뒤 AI로 대체 가능”', '성남시, 취업·이직 청년 AI역량 검사 등 ‘면접키트’ 5종 지원', '경주시, AI활용해 위기가구 발굴한다', '"AI 중심 재구성"…한국MS \'코파일럿+ PC 서피스 2종\' 예약 판매', '[해설]SKT, \'펭귄\' 업고 글로벌 AI데이터센터 공략…"AI생태계 완성"', '공공부문서 디지털트윈·AI기술 활용해 사회적 난제 해결한다', '최태원 회장 "SK, 생명과학부터 AI까지…혁신의 최전선"', '한컴, AI 명령어 템플릿으로 서울시 행정업무 효율화 제안', '경기교육청, 초·중 95개교서 AI 학습 프로그램 운영', '전자통신연구원, AI 안전성·신뢰성 높일 국제표준 제정', '"AI 기반 공항보안 CCTV 기술 협력"', "애플, 'iOS 18 베타' 일반 사용자에도 개방…AI는 없다", "'AI 수혜주' 전선업계에 무슨 일?…LS전선 “기술 탈취는 범죄” [뉴스+]", "'전력 먹는 하마' AI칩, 저전력 메모리가 뜬다", '“일자리 12%, AI 기술로 대체 가능성”', 'AI가 개발한 아이스크림 맛은?', "'슈퍼 박테리아' 무력화 후보물질 AI가 찾았다", '서울대 캠프·AI 강좌·가족 특강…강남구 여름방학 프로그램', '[현장] 전면에 나선 SPC家 차남 허희수 "BR 넘어 그룹 전반 AI

# 5. CSV 파일로 저장

In [ ]:

import pandas as pd


In [ ]:
#DF 변환
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})
news_df

In [ ]:
#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))


In [ ]:
# 저장
now = datetime.datetime.now()
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

중복 제거 후 행 개수:  28
